# Build your comparative framework!

In [6]:
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [197]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive


In [198]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import torch
import seaborn as sns
from matplotlib import axes
import os 
import pandas as pd
import json

import sys, os
sys.path.append('/cluster/home/kamara/Explain')
from clutils.nbutils import *
from clutils.nbutils.params import get_param_ranges

In [199]:
pal = sns.color_palette("Paired", 11)
dict_color = {"random":pal[0], "distance":pal[1], "pagerank":pal[2], "sa": pal[3], "ig": pal[4], "gradcam": pal[5], "occlusion":pal[6], 
              "basic_gnnexplainer": pal[7], "gnnexplainer": pal[8], 
              "pgmexplainer":pal[9], "subgraphx": pal[10]}


In [200]:
sns.set()
sns.set_context("notebook", rc={"legend.fontsize":20, "legend.title_fontsize":20, 
                                "axes.titlesize":19,"axes.labelsize":17,
                               "xtick.labelsize" : 16, "ytick.labelsize" : 16})


In [201]:
dir = '/cluster/home/kamara/Explain/csv/node_classification/real'

jsonpath = f'/cluster/home/kamara/Explain/configs/config_sparsity_real_1.json'
ranges = get_param_ranges(jsonpath)

df_sparsity = pd.read_csv(os.path.join(dir, f'nc_real_sparsity_1expe.csv'), index_col = 0)
res = df_sparsity.groupby(by=['dataset', 'sparsity', 'explainer_name', 'true_label_as_target', 'hard_mask']).mean()
res = res.reset_index()

In [202]:
def explainer_type(row):
    if row['explainer_name'] in ['basic_gnnexplainer', 'gnnexplainer', 'pgmexplainer', 'occlusion', 'subgraphx'] :
      return('pertubation')
    elif row['explainer_name'] in ['sa', 'ig', 'gradcam'] :
      return('gradient')
    return('baseline')

df_sparsity['type'] = df_sparsity.apply(lambda row: explainer_type(row), axis=1)

In [203]:
df_sparsity["1-fidelity_acc-"] = 1- df_sparsity["fidelity_acc-"]
df_sparsity["1-fidelity_prob-"] = 1- df_sparsity["fidelity_prob-"]
df_sparsity['charact_prob'] = 2 * df_sparsity['fidelity_prob+'] * (1-df_sparsity['fidelity_prob-'])/ (df_sparsity['fidelity_prob+']  + 1-df_sparsity['fidelity_prob-'])
df_sparsity['charact_acc'] = 2 * df_sparsity['fidelity_acc+'] * (1-df_sparsity['fidelity_acc-'])/ (df_sparsity['fidelity_acc+']  + 1-df_sparsity['fidelity_acc-'])


In [204]:
df_sparsity["1-fidelity_gnn_acc-"] = 1- df_sparsity["fidelity_gnn_acc-"]
df_sparsity["1-fidelity_gnn_prob-"] = 1- df_sparsity["fidelity_gnn_prob-"]
df_sparsity['charact_gnn_prob'] = 2 * df_sparsity['fidelity_gnn_prob+'] * (1-df_sparsity['fidelity_gnn_prob-'])/ (df_sparsity['fidelity_gnn_prob+']  + 1-df_sparsity['fidelity_gnn_prob-'])
df_sparsity['charact_gnn_acc'] = 2 * df_sparsity['fidelity_gnn_acc+'] * (1-df_sparsity['fidelity_gnn_acc-'])/ (df_sparsity['fidelity_gnn_acc+']  + 1-df_sparsity['fidelity_gnn_acc-'])


In [205]:
df_sparsity = df_sparsity.reset_index(drop=True)
df_sparsity

,dataset,true_label_as_target,hard_mask,sparsity,explainer_name,num_test,fidelity_acc+,fidelity_acc-,fidelity_prob+,fidelity_prob-,...,non_zero_values_init,type,1-fidelity_acc-,1-fidelity_prob-,charact_prob,charact_acc,1-fidelity_gnn_acc-,1-fidelity_gnn_prob-,charact_gnn_prob,charact_gnn_acc
0,actor,False,False,0.000,basic_gnnexplainer,100.0,-1.00,-1.00,-1.000000,-1.000000,...,1208.43,pertubation,2.00,2.000000,-4.000000,-4.000000,0.72,0.907846,0.267712,0.766753
1,actor,False,False,0.000,distance,100.0,-1.00,-1.00,-1.000000,-1.000000,...,60913.74,baseline,2.00,2.000000,-4.000000,-4.000000,0.21,0.838307,0.317123,0.345254
2,actor,False,False,0.000,gnnexplainer,100.0,-1.00,-1.00,-1.000000,-1.000000,...,1208.43,pertubation,2.00,2.000000,-4.000000,-4.000000,0.15,0.807601,0.319496,0.258716
3,actor,False,False,0.000,gradcam,100.0,-1.00,-1.00,-1.000000,-1.000000,...,39782.89,gradient,2.00,2.000000,-4.000000,-4.000000,0.26,0.838069,0.303979,0.398198
4,actor,False,False,0.000,ig,100.0,-1.00,-1.00,-1.000000,-1.000000,...,60916.10,gradient,2.00,2.000000,-4.000000,-4.000000,0.26,0.843095,0.310531,0.406387
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1434,wisconsin,True,True,0.997,occlusion,100.0,0.74,0.11,0.597992,0.027693,...,289.27,pertubation,0.89,0.972307,0.740537,0.808098,2.00,2.000000,-4.000000,-4.000000
1435,wisconsin,True,True,0.997,pagerank,100.0,0.28,0.07,0.166443,-0.064671,...,1085.55,baseline,0.93,1.064671,0.287881,0.430413,2.00,2.000000,-4.000000,-4.000000
1436,wisconsin,True,True,0.997,pgmexplainer,100.0,0.29,0.06,0.159620,-0.063297,...,540.31,pertubation,0.94,1.063297,0.277571,0.443252,2.00,2.000000,-4.000000,-4.000000
1437,wisconsin,True,True,0.997,random,100.0,0.24,0.89,0.112788,0.688925,...,1148.93,baseline,0.11,0.311075,0.165551,0.150857,2.00,2.000000,-4.000000,-4.000000


In [206]:
df = df_sparsity

In [207]:
sparsity_slider = widgets.SelectionSlider(
    options=['0', '0.7', '0.99', '0.997'],
    value='0',
    description='Sparsity:',
    disabled=False,
)



focus_select = widgets.ToggleButtons(
    options=[('Phenomenon', True), ('GNN model', False)],
    value = True,
    description='Focus:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Explain phenomenon', 'Explain GNN logic'],
#     icons=['check'] * 3
)

hard_mask_select = widgets.ToggleButtons(
    options=[('Hard', True), ('Soft', False)],
    value = True,
    description='Nature:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Mask has binary values', 'Mask has scalar values'],
#     icons=['check'] * 3
)

charact_type = widgets.ToggleButtons(
    options=[('Closeness', True), ('Reproducibility', False)],
    value = True,
    description='Characterization:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Characterization on probability', 'Characterization on accuracy'],
#     icons=['check'] * 3
)


dataset_select = widgets.Dropdown(
    options=["cora",
      "pubmed",
      "citeseer",
      "facebook",
      "cornell",
      "texas",
      "wisconsin",
      "actor",
      "chameleon",
      "squirrel"],
    value='cora',
    description='Dataset:',
    disabled=False,
)


explainer_select = widgets.Dropdown(
    options=["random",
      "distance",
      "pagerank",
      "sa",
      "ig",
      "gradcam",
      "occlusion",
      "basic_gnnexplainer",
      "gnnexplainer",
      "pgmexplainer"],
    value='random',
    description='Explainer:',
    disabled=False,
)


In [208]:
def reduce(df, **kwargs):
    if 'focus_phen' in kwargs:
        focus_phen = kwargs.get('focus_phen')
        df = df.loc[df['true_label_as_target'] == focus_phen]
        prob = kwargs.get('prob') if 'prob' in kwargs else True
        if focus_phen:
            df.loc[:, "charact"] = df.loc[:,"charact_prob"] if prob else df.loc[:,"charact_acc"]
            df.loc[:, "fid+"] = df.loc[:,"fidelity_prob+"] if prob else df.loc[:,"fidelity_acc+"]
            df.loc[:, "1-fid-"] = df.loc[:,"1-fidelity_prob-"] if prob else df.loc[:,"1-fidelity_acc-"]
        else:
            df.loc[:, "charact"] = df.loc[:, "charact_gnn_prob"] if prob else df.loc[:, "charact_gnn_acc"]
            df.loc[:, "fid+"] = df.loc[:, "fidelity_gnn_prob+"] if prob else df.loc[:, "fidelity_gnn_acc+"]
            df.loc[:, "1-fid-"] = df.loc[:, "1-fidelity_gnn_prob-"] if prob else df.loc[:, "1-fidelity_gnn_acc-"]
    
    if 'hard_mask' in kwargs:
        df = df.loc[df['hard_mask'] == kwargs.get('hard_mask')]
    if 'sparsity' in kwargs:
        df = df.loc[df['sparsity'] == eval(kwargs.get('sparsity'))]
    if 'dataset' in kwargs:
        df = df.loc[df['dataset'] == kwargs.get('dataset')]
    if 'explainer' in kwargs:
        df = df.loc[df['explainer_name'] == kwargs.get('explainer')]
    return df

In [193]:
@interact
def score(focus_phen=focus_select, hard_mask=hard_mask_select, sparsity=sparsity_slider, prob=charact_type):
    dfs = reduce(df, focus_phen=focus_phen, prob=prob, hard_mask=hard_mask, sparsity=sparsity)
    df_char = pd.pivot_table(dfs, values='charact', index=['explainer_name'], columns='dataset')
    df_char['mean_charact'] = df_char.mean(axis=1)
    
    res = df_char.reset_index().rename_axis(None, axis=1)
    sns.barplot(x="mean_charact", y="explainer_name", data=res, palette=dict_color)
    
    df_char = df_char.sort_values(by=['mean_charact'])
    return df_char

interactive(children=(ToggleButtons(description='Focus:', options=(('Phenomenon', True), ('GNN model', False))…

In [159]:
# Mask properties
@interact
def mask_properties(dataset=dataset_select, focus_phen=focus_select, hard_mask=hard_mask_select, prob=charact_type, sparsity=sparsity_slider):
    df_mask = reduce(df, focus_phen=focus_phen, prob=prob, hard_mask=hard_mask, sparsity=sparsity, dataset=dataset)
    df_mask = df_mask[["explainer_name", "mask_size_transf", "mask_entropy_transf", "max_avg_transf", "charact"]]
    df_mask = df_mask.sort_values(by=['charact'])
    return df_mask.set_index('explainer_name')

interactive(children=(Dropdown(description='Dataset:', options=('cora', 'pubmed', 'citeseer', 'facebook', 'cor…

In [195]:
@interact
def get_explanation(focus_phen=focus_select, hard_mask=hard_mask_select, sparsity=sparsity_slider, prob=charact_type, dataset=dataset_select, explainer=explainer_select):
    df_unit = reduce(df, focus_phen=focus_phen, prob=prob, hard_mask=hard_mask, sparsity=sparsity, dataset=dataset, explainer=explainer)
    # Causal: df_unit['1-fid-']
    # Counterfactual: df_unit['fid+']
    # Characterization: df_unit['charact']
    ax = sns.barplot(x="value", y="variable", data=pd.melt(df_unit[['1-fid-', 'fid+', 'charact']]), palette=sns.color_palette("tab10"))
    plt.xlim(0,1)
    ax.set_yticklabels(['causal','counterfactual', 'characterization'])
    ax.set(xlabel=None)
    ax.set(ylabel=None)
    ax.set(title="Type of explanation")
    

interactive(children=(ToggleButtons(description='Focus:', index=1, options=(('Phenomenon', True), ('GNN model'…

In [213]:
@interact
def get_explanation(focus_phen=focus_select, hard_mask=hard_mask_select, prob=charact_type):
    dfs = reduce(df, focus_phen=focus_phen, prob=prob, hard_mask=hard_mask)
    # Causal: df_unit['1-fid-']
    # Counterfactual: df_unit['fid+']
    # Characterization: df_unit['charact']
    g = sns.relplot(
        data=dfs,
        x="fid+", y="1-fid-", col="sparsity", row="dataset", 
        hue="explainer_name", palette=dict_color, s=140, style="type", alpha =1, edgecolor="grey"
    )
    
    plt.xlim(0, 1)
    plt.ylim(0, 1.2)
    for lh in g._legend.legendHandles: 
        lh.set_alpha(1)
        lh._sizes = [130] 

interactive(children=(ToggleButtons(description='Focus:', options=(('Phenomenon', True), ('GNN model', False))…